In [1]:
import polars as pl
import polars.selectors as cs

In [2]:
df = pl.read_ndjson('../data/ledgers/plata.ndjson')

In [3]:
from zlib import crc32

def assign_split(column: str) -> pl.Expr:

    seed = (
        pl.col(column)
        .cast(pl.String)
        .map_elements(lambda x: float(crc32(str.encode(x)) & 0xFFFFFFFF), return_dtype=pl.Float32)
        .mul(1 / 2**32)
    )

    return (
        pl.when(seed.is_between(0.0, 0.6))
        .then(pl.lit("train"))
        .when(seed.is_between(0.6, 0.8))
        .then(pl.lit("validate"))
        .when(seed.is_between(0.8, 1.0))
        .then(pl.lit("test"))
        .otherwise(pl.lit("train"))
    )

In [4]:
dynamic = (
    df.select(
        "bureau_search_ref",
        pl.col("created_at").str.replace("\+00", "").str.to_date(format="%Y-%m-%d %H:%M:%S%.f"),
        cs.by_dtype(pl.List(pl.String)) | cs.by_dtype(pl.List(pl.Int64)),
        target=pl.col("irb"),
    )
    .explode(pl.exclude("created_at","bureau_search_ref", "target"))
    .select(
        cs.all() - cs.string() - cs.by_name("account_index"),
        (cs.string() - cs.by_name("m")).replace("", None),
        statement_id=pl.concat_str("accgroupid", pl.lit("$"), "m"),
        account_open=pl.col("accstartdate").str.to_date(),
        account_index=pl.col("accgroupid").cast(pl.String),
        statement_date=pl.col("m").str.to_date(format="%Y-%m"),
    )
    # filter out statements of accounts with empty balances
    .filter(pl.col("bal") > 0)
    .select(
        "payamt", "bal", "history_limit", "bureau_search_ref", "pay", "acctypecode", "target", "statement_id", "account_index",
        # calculate two duration fields (time diff between statement and application date, statement and account open date)
        statement_tenure=(pl.col("statement_date") - pl.col("created_at")).dt.total_days(),
        account_tenure=(pl.col("account_open") - pl.col("created_at")).dt.total_days(),
    )
    # .select(
    #     pl.exclude("payamt", "history_limit"),
    #     pl.col("payamt").fill_null(-1),
    #     pl.col("history_limit").fill_null(-1),
    # )
    .sort("bureau_search_ref", "statement_tenure")
    .group_by("bureau_search_ref")
    .agg(cs.all())
)

In [5]:
dynamic.describe()

shape: (9, 12)
┌────────────┬────────────┬────────┬────────┬───┬────────────┬────────────┬────────────┬───────────┐
│ statistic  ┆ bureau_sea ┆ payamt ┆ bal    ┆ … ┆ statement_ ┆ account_in ┆ statement_ ┆ account_t │
│ ---        ┆ rch_ref    ┆ ---    ┆ ---    ┆   ┆ id         ┆ dex        ┆ tenure     ┆ enure     │
│ str        ┆ ---        ┆ f64    ┆ f64    ┆   ┆ ---        ┆ ---        ┆ ---        ┆ ---       │
│            ┆ str        ┆        ┆        ┆   ┆ f64        ┆ f64        ┆ f64        ┆ f64       │
╞════════════╪════════════╪════════╪════════╪═══╪════════════╪════════════╪════════════╪═══════════╡
│ count      ┆ 4542       ┆ 4542.0 ┆ 4542.0 ┆ … ┆ 4542.0     ┆ 4542.0     ┆ 4542.0     ┆ 4542.0    │
│ null_count ┆ 0          ┆ 0.0    ┆ 0.0    ┆ … ┆ 0.0        ┆ 0.0        ┆ 0.0        ┆ 0.0       │
│ mean       ┆ null       ┆ null   ┆ null   ┆ … ┆ null       ┆ null       ┆ null       ┆ null      │
│ std        ┆ null       ┆ null   ┆ null   ┆ … ┆ null       ┆ null       ┆ null       ┆ null      │
│ min        ┆ BS43GGXB   ┆ null   ┆ null   ┆ … ┆ null       ┆ null       ┆ null       ┆ null      │
│ 25%        ┆ null       ┆ null   ┆ null   ┆ … ┆ null       ┆ null       ┆ null       ┆ null      │
│ 50%        ┆ null       ┆ null   ┆ null   ┆ … ┆ null       ┆ null       ┆ null       ┆ null      │
│ 75%        ┆ null       ┆ null   ┆ null   ┆ … ┆ null       ┆ null       ┆ null       ┆ null      │
│ max        ┆ BS45AMVB   ┆ null   ┆ null   ┆ … ┆ null       ┆ null       ┆ null       ┆ null      │
└────────────┴────────────┴────────┴────────┴───┴────────────┴────────────┴────────────┴───────────┘

In [6]:
static = (
    df.select(
        (cs.string() - cs.by_name('loan_account_ref')).replace("", None),
        # cs.integer().fill_null(-1),
        cs.integer(),
        split=assign_split('bureau_search_ref'),
    )
)

In [7]:
static.describe()

shape: (9, 82)
┌────────────┬───────────────────┬──────┬──────┬───┬───────────┬───────────┬───────────┬──────────┐
│ statistic  ┆ bureau_search_ref ┆ ibc  ┆ nob  ┆ … ┆ gsb       ┆ xe        ┆ fb        ┆ split    │
│ ---        ┆ ---               ┆ ---  ┆ ---  ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ str        ┆ str               ┆ str  ┆ str  ┆   ┆ f64       ┆ f64       ┆ f64       ┆ str      │
╞════════════╪═══════════════════╪══════╪══════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ count      ┆ 4542              ┆ 1177 ┆ 3982 ┆ … ┆ 4101.0    ┆ 4033.0    ┆ 4542.0    ┆ 4542     │
│ null_count ┆ 0                 ┆ 3365 ┆ 560  ┆ … ┆ 441.0     ┆ 509.0     ┆ 0.0       ┆ 0        │
│ mean       ┆ null              ┆ null ┆ null ┆ … ┆ 11.073884 ┆ 83.449293 ┆ 11.325407 ┆ null     │
│ std        ┆ null              ┆ null ┆ null ┆ … ┆ 5.65566   ┆ 81.735212 ┆ 4.999781  ┆ null     │
│ min        ┆ BS43GGXB          ┆ 0    ┆ 1    ┆ … ┆ 1.0       ┆ 0.0       ┆ 1.0       ┆ test     │
│ 25%        ┆ null              ┆ null ┆ null ┆ … ┆ 6.0       ┆ 27.0      ┆ 8.0       ┆ null     │
│ 50%        ┆ null              ┆ null ┆ null ┆ … ┆ 11.0      ┆ 58.0      ┆ 11.0      ┆ null     │
│ 75%        ┆ null              ┆ null ┆ null ┆ … ┆ 16.0      ┆ 115.0     ┆ 14.0      ┆ null     │
│ max        ┆ BS45AMVB          ┆ U    ┆ XX   ┆ … ┆ 20.0      ┆ 497.0     ┆ 47.0      ┆ validate │
└────────────┴───────────────────┴──────┴──────┴───┴───────────┴───────────┴───────────┴──────────┘

In [8]:
static.select(cs.all() - cs.string()).describe()

shape: (9, 68)
┌───────────┬──────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ statistic ┆ peb      ┆ wm        ┆ bcc       ┆ … ┆ pob       ┆ gsb       ┆ xe        ┆ fb        │
│ ---       ┆ ---      ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ str       ┆ f64      ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
╞═══════════╪══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ count     ┆ 4542.0   ┆ 4482.0    ┆ 3060.0    ┆ … ┆ 4463.0    ┆ 4101.0    ┆ 4033.0    ┆ 4542.0    │
│ null_coun ┆ 0.0      ┆ 60.0      ┆ 1482.0    ┆ … ┆ 79.0      ┆ 441.0     ┆ 509.0     ┆ 0.0       │
│ t         ┆          ┆           ┆           ┆   ┆           ┆           ┆           ┆           │
│ mean      ┆ 0.004844 ┆ 145.07474 ┆ 26.156209 ┆ … ┆ 162.91351 ┆ 11.073884 ┆ 83.449293 ┆ 11.325407 │
│           ┆          ┆ 3         ┆           ┆   ┆ 1         ┆           ┆           ┆           │
│ std       ┆ 0.083806 ┆ 364.91922 ┆ 21.028797 ┆ … ┆ 462.13210 ┆ 5.65566   ┆ 81.735212 ┆ 4.999781  │
│           ┆          ┆ 8         ┆           ┆   ┆ 9         ┆           ┆           ┆           │
│ min       ┆ 0.0      ┆ 0.0       ┆ 0.0       ┆ … ┆ 0.0       ┆ 1.0       ┆ 0.0       ┆ 1.0       │
│ 25%       ┆ 0.0      ┆ 92.0      ┆ 7.0       ┆ … ┆ 73.0      ┆ 6.0       ┆ 27.0      ┆ 8.0       │
│ 50%       ┆ 0.0      ┆ 98.0      ┆ 21.0      ┆ … ┆ 92.0      ┆ 11.0      ┆ 58.0      ┆ 11.0      │
│ 75%       ┆ 0.0      ┆ 107.0     ┆ 44.0      ┆ … ┆ 121.0     ┆ 16.0      ┆ 115.0     ┆ 14.0      │
│ max       ┆ 2.0      ┆ 9494.0    ┆ 72.0      ┆ … ┆ 9700.0    ┆ 20.0      ┆ 497.0     ┆ 47.0      │
└───────────┴──────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴───────────┘

In [9]:
ledger = dynamic.join(static, on='bureau_search_ref')

In [10]:
out = {}

for field, dtype in dict(static.schema).items():
    out[field] = "category" if dtype == pl.String else "number"

In [11]:
static['nob'].unique().to_list()

['22',
 '18',
 '16',
 '13',
 '24',
 '12',
 '07',
 None,
 '01',
 '14',
 'XX',
 '11',
 '04',
 '21',
 '09',
 '20',
 '05',
 '23',
 '15',
 '06',
 '02',
 '10',
 '19',
 '03',
 '08',
 '17']

In [12]:
for index, shard in enumerate(ledger.iter_slices(10)):
    shard.write_avro(f"../data/lifestreams/plata/plata-{index}.avro", name="plata")

In [13]:
dynamic.select(
    acctypecode=pl.col('bal').list.len()
).get_column('acctypecode').plot.hist()

:Histogram   [acctypecode]   (Count)